In [1]:
!nvidia-smi

Mon Aug 28 20:02:03 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.66                 Driver Version: 375.66                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:04:00.0     Off |                  N/A |
| 27%   33C    P8     9W / 180W |    434MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:05:00.0     Off |                  N/A |
| 27%   35C    P8     9W / 180W |      2MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
%matplotlib notebook
import importlib
import numpy as np
import pandas as pd
import keras
from keras.layers import TimeDistributed, Dense, Dropout, GRU, Bidirectional
from keras.models import Sequential
from tracker import extractor as ext, utils, metrics, visuals

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:06:00.0)


In [3]:
%%time
modelpath   = "data/models/UNIF-10N-25T-200E-235R-RZ.h5"
trainpath   = "data/sets/UNIF-10N-25T-200E-235R.gz"
testpath    = "data/sets/RAMP-10N-25T-235R.gz"
train_frame = pd.read_csv(trainpath)
test_frame  = pd.read_csv(testpath)
train = utils.list_of_groups(train_frame, group="event_id")
test  = utils.list_of_groups(test_frame,  group="event_id")
print("Train is list of {} events.".format(len(train)))
print("Test is list of {} events.".format(len(test)))
if (not utils.is_prepared(train_frame)) or (not utils.is_prepared(test_frame)):
    print("Warning: frame is not prepared.")
    print("Look at the prepare_frame() function in tracker/extractor.py")   

Train is list of 4800 events.
Test is list of 3600 events.
CPU times: user 2.96 s, sys: 60 ms, total: 3.02 s
Wall time: 3.02 s


In [4]:
order = ["phi", "r", "z"]

In [6]:
input_shape  = (235, 2)
n_categories = 25 + 2
optimizer    = keras.optimizers.RMSprop(lr=0.001)
histories    = []

In [7]:
model = Sequential()
model.add(Bidirectional(
    GRU(units=256, return_sequences=True, recurrent_dropout=1/2, implementation=2),
    merge_mode="mul",
    input_shape=input_shape))
model.add(Dropout(rate=1/2))
model.add(Bidirectional(
    GRU(units=256, return_sequences=True, recurrent_dropout=1/2, implementation=2),
    merge_mode="mul"))
model.add(Dropout(rate=1/2))
model.add(Bidirectional(
    GRU(units=256, return_sequences=True, recurrent_dropout=1/2, implementation=2),
    merge_mode="mul"))
model.add(Dropout(rate=1/2))
model.add(TimeDistributed(Dense(units=n_categories, kernel_initializer="uniform", activation="softmax")))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 235, 256)          397824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 235, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 235, 256)          787968    
_________________________________________________________________
dropout_2 (Dropout)          (None, 235, 256)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 235, 256)          787968    
_________________________________________________________________
dropout_3 (Dropout)          (None, 235, 256)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 235, 27)           6939      
Total para

In [14]:
%%time
# L1 = np.array([ext.extract_input(a, order)[:, 1:] for a in train])
# L2 = np.array([ext.extract_input(a, order)[:, 1:] for a in test])
# T1 = np.array([ext.extract_output(a, order) for a in train])
# T2 = np.array([ext.extract_output(a, order) for a in test])


epochs     = 32
batch_size = 100
histories.append(model.fit(
    L1, T1,
    batch_size=64,
    epochs=epochs,
    verbose=1,
    validation_data=(L2, T2)))

Train on 4800 samples, validate on 3600 samples
Epoch 1/32
4800/4800 [==============================] - 76s - loss: 0.8023 - acc: 0.6997 - val_loss: 0.4971 - val_acc: 0.8003
Epoch 2/32
4800/4800 [==============================] - 76s - loss: 0.7813 - acc: 0.7076 - val_loss: 0.5096 - val_acc: 0.7982
Epoch 3/32
4800/4800 [==============================] - 76s - loss: 0.7642 - acc: 0.7140 - val_loss: 0.5194 - val_acc: 0.7914
Epoch 4/32
4800/4800 [==============================] - 77s - loss: 0.7498 - acc: 0.7205 - val_loss: 0.4400 - val_acc: 0.8343
Epoch 5/32
4800/4800 [==============================] - 77s - loss: 0.7340 - acc: 0.7243 - val_loss: 0.5268 - val_acc: 0.7899
Epoch 6/32
4800/4800 [==============================] - 77s - loss: 0.7223 - acc: 0.7306 - val_loss: 0.4366 - val_acc: 0.8338
Epoch 7/32
4800/4800 [==============================] - 77s - loss: 0.7096 - acc: 0.7328 - val_loss: 0.3986 - val_acc: 0.8615
Epoch 8/32
4800/4800 [==============================] - 77s - loss: 0.